In [22]:
#!/usr/bin/env python
# coding: utf-8

In [23]:
import requests
from datetime import datetime, timezone

# 1️⃣ Use today's UTC date in the filename
today_str = datetime.now(timezone.utc).strftime("%Y%m%d")
tle_file = f'tle_data_{today_str}.txt'

# 2️⃣ CelesTrak's TLE for "Active Satellites" (updated and working)
url = 'https://celestrak.com/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'

# 3️⃣ Fetch and save TLE data
response = requests.get(url)
if response.status_code == 200:
    with open(tle_file, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print(f"✅ TLE data downloaded and saved to: {tle_file}")
else:
    print(f"❌ Failed to download TLE data. Status code: {response.status_code}")

✅ TLE data downloaded and saved to: tle_data_20250626.txt


In [24]:
#Load saved TLE file and compute satellite geolocations

from skyfield.api import load, wgs84
from datetime import datetime, timezone
import pandas as pd
import os

# 1️⃣ Generate today's TLE file name
today_str = datetime.now(timezone.utc).strftime("%Y%m%d")
tle_file = f'tle_data_{today_str}.txt'

# 2️⃣ Check if the TLE file exists
if not os.path.exists(tle_file):
    raise FileNotFoundError(f"TLE file {tle_file} not found. Please run Step 1 first.")

# 3️⃣ Load satellites from saved TLE file
satellites = load.tle_file(tle_file)
print(f"📦 Loaded {len(satellites)} satellites")

# 4️⃣ Get current UTC time as Skyfield time object
ts = load.timescale()
now = ts.now()

# 5️⃣ Compute ground subpoints for first N satellites
records = []
for sat in satellites[:20]:
    try:
        geocentric = sat.at(now)
        subpoint = wgs84.subpoint(geocentric)

        records.append({
            'name': sat.name,
            'latitude': subpoint.latitude.degrees,
            'longitude': subpoint.longitude.degrees,
            'altitude_km': subpoint.elevation.km,
            'dist_from_equator': abs(subpoint.latitude.degrees)
        })
    except Exception as e:
        print(f"⚠️ Skipping {sat.name} due to error: {e}")

# ✅ Now create DataFrame *after* loop, not inside try/except
df = pd.DataFrame(records)
print("📊 Sample satellite data:")
print(df.head())

📦 Loaded 11858 satellites
📊 Sample satellite data:
           name   latitude   longitude  altitude_km  dist_from_equator
0   CALSPHERE 1  31.275885   94.845131   983.754616          31.275885
1   CALSPHERE 2  40.032461  -80.977221  1056.224984          40.032461
2         LCS 1  32.142316   69.108034  2798.807324          32.142316
3     TEMPSAT 1 -27.378914   64.307934  1103.268944          27.378914
4  CALSPHERE 4A -17.145499  156.992796  1093.505692          17.145499


In [25]:
# Compute Satellite Position Features (Lat, Lon, Altitude)

from skyfield.api import wgs84
from datetime import datetime, timezone

def compute_satellite_features(satellites, ts):
    """
    Computes geodetic features (lat, lon, altitude) for each satellite.
    Parameters:
        satellites: list of skyfield EarthSatellite objects
        ts: Skyfield TimeScale object
    Returns:
        DataFrame with satellite name, latitude, longitude, altitude_km, distance from equator
    """
    now = ts.now()  # ✅ Must use Skyfield Time object
    data = []

    for sat in satellites:
        try:
            geocentric = sat.at(now)  # This requires a Time object
            subpoint = wgs84.subpoint(geocentric)
            lat = subpoint.latitude.degrees
            lon = subpoint.longitude.degrees
            alt = subpoint.elevation.km
            dist_from_equator = abs(lat)

            data.append({
                'name': sat.name,
                'latitude': lat,
                'longitude': lon,
                'altitude_km': alt,
                'dist_from_equator': dist_from_equator
            })
        except Exception as e:
            print(f"⚠️ Failed for {sat.name}: {e}")

    df = pd.DataFrame(data)
    return df

# ✅ Usage Example:
df_features = compute_satellite_features(satellites, ts)
print("📊 Sample satellite features:")
print(df_features.head())

📊 Sample satellite features:
           name   latitude   longitude  altitude_km  dist_from_equator
0   CALSPHERE 1  31.277858   94.844977   983.755551          31.277858
1   CALSPHERE 2  40.030517  -80.977378  1056.224298          40.030517
2         LCS 1  32.142246   69.109556  2798.807443          32.142246
3     TEMPSAT 1 -27.380846   64.307792  1103.270807          27.380846
4  CALSPHERE 4A -17.143562  156.992656  1093.503805          17.143562


In [26]:
#Classify Risk Levels Based on Satellite Features

def classify_risk_levels(df):
    """
    Adds a 'risk_level' column based on distance from equator.
    Thresholds:
        - Low: <= 10 degrees
        - Medium: 10–50 degrees
        - High: > 50 degrees
    Parameters:
        df: DataFrame with satellite features (including 'dist_from_equator')
    Returns:
        Updated DataFrame with a new 'risk_level' column
    """
    def risk_category(dist):
        if dist <= 10:
            return 'Low'
        elif dist <= 50:
            return 'Medium'
        else:
            return 'High'

    df['risk_level'] = df['dist_from_equator'].apply(risk_category)
    return df

# ✅ Apply risk classification to satellite features
df_features = classify_risk_levels(df_features)

print("🛰️ Satellite features with risk levels:")
print(df_features[['name', 'latitude', 'longitude', 'altitude_km', 'risk_level']].head())


# ### 🚨 Risk Level Classification Based on Distance from Equator
# 
# This section assigns a `risk_level` to each satellite using its distance from the equator (in degrees of latitude). This is used as a proxy to assess potential conjunction risk or exposure to debris and traffic.
# 
# **Thresholds:**
# - **Low Risk** (≤ 10°): Equatorial or near-equatorial orbits. Lower orbital traffic.
# - **Medium Risk** (10°–50°): Moderately inclined orbits. Moderate intersection probability.
# - **High Risk** (> 50°): Polar or sun-synchronous orbits. Highest chance of orbital crossings.
# 
# > 🔍 Note: This is a basic heuristic. It can later be improved by integrating actual debris maps, relative velocities, or orbital crossings.
#

🛰️ Satellite features with risk levels:
           name   latitude   longitude  altitude_km risk_level
0   CALSPHERE 1  31.277858   94.844977   983.755551     Medium
1   CALSPHERE 2  40.030517  -80.977378  1056.224298     Medium
2         LCS 1  32.142246   69.109556  2798.807443     Medium
3     TEMPSAT 1 -27.380846   64.307792  1103.270807     Medium
4  CALSPHERE 4A -17.143562  156.992656  1093.503805     Medium


In [27]:
# Plot Satellite Subpoints and Risk Levels Using Plotly
import plotly.io as pio
pio.renderers.default = 'vscode'

import pandas as pd


# 📍 Function to plot the satellite risk map
def plot_satellite_risk_map(df, sample_size=50):
    import plotly.graph_objects as go

    # Randomly sample satellites to avoid clutter
    df_sampled = df.sample(n=min(sample_size, len(df)))

    # Assign colors based on risk level
    color_map = {'Low': 'green', 'Medium': 'orange', 'High': 'red'}
    df_sampled['color'] = df_sampled['risk_level'].map(color_map)

    fig = go.Figure(
        go.Scattergeo(
            lon = df_sampled['longitude'],
            lat = df_sampled['latitude'],
            text = df_sampled['name'] + '<br>Risk: ' + df_sampled['risk_level'],
            mode = 'markers+text',
            marker = dict(
                size = 6,
                color = df_sampled['color'],
                line = dict(width=0.5, color='black'),
                opacity = 0.8
            ),
            textfont = dict(
                size = 8,
                color = 'black'
            ),
            textposition = 'top center'
        )
    )

    fig.update_geos(showland=True, landcolor="LightGray")
    fig.update_layout(
        title='🛰️ Satellite Ground Track Risk Map',
        geo=dict(
            projection_type='orthographic',
            showcoastlines=True,
            showland=True,
            showocean=True,
            oceancolor='lightblue',
            landcolor='beige',
            showframe=False,
        )
    )

    return fig  # ✅ important

fig = plot_satellite_risk_map(df_features, sample_size=50)
# Save the HTML to docs
save_path = os.path.join(os.getcwd(), "docs", "satellite_risk_map_sample.html")
fig.write_html(save_path)

fig.show()

In [28]:
import os
from datetime import datetime

# ✅ Ensure folder exists
output_dir = "visualizations/risk_maps"
os.makedirs(output_dir, exist_ok=True)

# 📅 Create dated filenames
today_str = datetime.today().strftime("%Y-%m-%d")
html_path = f"{output_dir}/{today_str}_satellite_risk_map.html"
csv_path = f"{output_dir}/{today_str}_risk_data.csv"

# ✅ Save files
fig.write_html(html_path)
df_features.to_csv(csv_path, index=False)

print(f"✅ Saved: {html_path}")
print(f"✅ Saved: {csv_path}")

✅ Saved: visualizations/risk_maps/2025-06-26_satellite_risk_map.html
✅ Saved: visualizations/risk_maps/2025-06-26_risk_data.csv
